## A partir daqui você deve alterar seu notebook conforme orientações da atividade 10 do enunciado do trabalho prático.

Nas células abaixo foram deixadas algumas dicas para você seguir.

In [7]:
import mysql.connector
import json
import requests
from datetime import datetime

In [2]:
response = requests.get("https://api-coleta-dados.herokuapp.com/api/pessoas")

In [3]:
print(response.status_code)

200


In [5]:
pessoas = []
if response.status_code == 200:
    pessoas = json.loads(response.content)

In [8]:
for pessoa in pessoas:
	    if pessoa['idade'] == None:
	        dataNascimento = datetime.strptime(pessoa['data_nasc'], '%d/%m/%Y')
	        anoCorrente = datetime.now().year
	        
	        ano = dataNascimento.year
	        idade = anoCorrente - ano
	        pessoa['idade'] = idade
	        # pessoa['data_nasc'] = dataNascimento
	        print('Nome: %s  - Data Nascimento: %s - Idade: %s' % (pessoa['nome'], pessoa['data_nasc'], pessoa['idade']))
	        
	    if len(pessoa['data_nasc']) < 10:
	        anoCorrente = datetime.now().year
	        dataNascimento = pessoa['data_nasc'] + '/' + str(anoCorrente - pessoa['idade'])
	        # dataNascimento = datetime.strptime(dataNascimento, '%d/%m/%Y')
	        pessoa['data_nasc'] = str(dataNascimento)
	        print('Nome: %s - Data Nascimento: %s - Idade: %s' % (pessoa['nome'], pessoa['data_nasc'], pessoa['idade']))
        

Nome: Silvana Hadassa  - Data Nascimento: 12/06/1964 - Idade: 56
Nome: Cauã Henrique - Data Nascimento: 09/03/1970 - Idade: 50
Nome: Bianca Flávia  - Data Nascimento: 23/01/1949 - Idade: 71
Nome: Arthur Filipe  - Data Nascimento: 23/12/1987 - Idade: 33
Nome: Teresinha Jaqueline  - Data Nascimento: 20/10/1968 - Idade: 52
Nome: Jorge Nicolas  - Data Nascimento: 11/07/1968 - Idade: 52
Nome: Daniel Lucca - Data Nascimento: 22/01/1999 - Idade: 21
Nome: João Pedro  - Data Nascimento: 08/05/2002 - Idade: 18
Nome: Antonio Sérgio - Data Nascimento: 15/04/1971 - Idade: 49
Nome: Thales Bento  - Data Nascimento: 18/04/1942 - Idade: 78
Nome: Carolina Márcia  - Data Nascimento: 25/11/1959 - Idade: 61
Nome: Levi Sérgio  - Data Nascimento: 21/08/1949 - Idade: 71
Nome: Rafael Ryan - Data Nascimento: 10/01/1950 - Idade: 70
Nome: Martin Edson - Data Nascimento: 15/01/1985 - Idade: 35
Nome: Isadora Sueli  - Data Nascimento: 03/07/1999 - Idade: 21
Nome: Bruno Manuel - Data Nascimento: 14/09/1963 - Idade: 5

In [9]:
estados = set()
tipos_sanguineos = set()
cores = set()

In [10]:
estados = [pessoa['estado'] for pessoa in pessoas 
    if pessoa['estado'] not in estados and not estados.add(pessoa['estado'])]

In [11]:
tipos_sanguineos = [pessoa['tipo_sanguineo'] for pessoa in pessoas 
    if pessoa['tipo_sanguineo'] not in tipos_sanguineos and not tipos_sanguineos.add(pessoa['tipo_sanguineo'])]

In [12]:
cores = [pessoa['cor'] for pessoa in pessoas 
    if pessoa['cor'] not in cores and not cores.add(pessoa['cor'])]

In [13]:
print(estados)
print(tipos_sanguineos)
print(cores) 

['DF', 'MS', 'AP', 'MG', 'MT', 'PE', 'AC', 'TO', 'AM', 'RO', 'GO', 'RS', 'RR', 'MA', 'ES', 'PI', 'PR', 'AL', 'SE', 'PA', 'BA', 'RJ', 'PB', 'SC', 'CE', 'RN']
['A+', 'B+', 'AB-', 'O+', 'O-', 'AB+', 'A-', 'B-']
['vermelho', 'laranja', 'verde', 'preto', 'roxo', 'azul', 'amarelo']


In [17]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='igti',
    database='db_desafio'
)

print(mydb)
cursor = mydb.cursor()

In [59]:
for estado in estados:        
    query = "INSERT IGNORE INTO tb_estado (sigla) values ('%s')" % estado
    cursor.execute(query)
    mydb.commit()

In [60]:
for tipo_sanguineo in tipos_sanguineos:        
    query = "INSERT IGNORE INTO tb_tiposanguineo (tipo) values ('%s')" % tipo_sanguineo
    cursor.execute(query)
    mydb.commit()

In [61]:
for cor in cores:        
    query = "INSERT IGNORE INTO tb_cor (cor) values ('%s')" % cor
    cursor.execute(query)
    mydb.commit()

In [62]:
cidades = set()

cidades = [(pessoa['cidade'], pessoa['estado']) for pessoa in pessoas if pessoa['cidade'] not in cidades and not cidades.add(pessoa['cidade'])]

print(cidades)

[('Brasília', 'DF'), ('Campo Grande', 'MS'), ('Macapá', 'AP'), ('Contagem', 'MG'), ('Rondonópolis', 'MT'), ('Olinda', 'PE'), ('Rio Branco', 'AC'), ('Araguaína', 'TO'), ('Manaus', 'AM'), ('Caruaru', 'PE'), ('Porto Velho', 'RO'), ('Planaltina', 'GO'), ('Santa Cruz do Sul', 'RS'), ('Porto Alegre', 'RS'), ('Carpina', 'PE'), ('Boa Vista', 'RR'), ('Imperatriz', 'MA'), ('São Luís', 'MA'), ('Cacoal', 'RO'), ('Cariacica', 'ES'), ('Parnaíba', 'PI'), ('Santana', 'AP'), ('Ponta Grossa', 'PR'), ('Arapiraca', 'AL'), ('Águas Lindas de Goiás', 'GO'), ('Cianorte', 'PR'), ('Aracaju', 'SE'), ('Belém', 'PA'), ('Jataí', 'GO'), ('Várzea Grande', 'MT'), ('Maceió', 'AL'), ('Salvador', 'BA'), ('Macaé', 'RJ'), ('Cuiabá', 'MT'), ('Santa Luzia do Itanhy', 'SE'), ('João Pessoa', 'PB'), ('Brusque', 'SC'), ('Rio Grande', 'RS'), ('Juazeiro do Norte', 'CE'), ('Natal', 'RN'), ('Londrina', 'PR')]


In [64]:
for cidade in cidades:
    query_cidade_exists = "SELECT COUNT(*) FROM tb_cidade WHERE nome = '%s'" % cidade[0]
    cursor.execute(query_cidade_exists)
    qtd_cidade = cursor.fetchone()[0]
    
    if qtd_cidade == 0:
        queryestado = "SELECT id FROM tb_estado WHERE sigla = '%s'" % cidade[1]
        cursor.execute(queryestado)
        id_estado = cursor.fetchone()[0]
 
        querycidade = "INSERT INTO tb_cidade (nome, id_estado) VALUES ('%s', %s) " % (cidade[0], id_estado)
    
        cursor.execute(querycidade)
        mydb.commit() 

In [67]:
for pessoa in pessoas:
    querycidade = "SELECT id FROM tb_cidade WHERE nome = '%s'" % pessoa['cidade']
    cursor.execute(querycidade)
    id_cidade = cursor.fetchone()[0]
    
    querycor = "SELECT id FROM tb_cor WHERE cor = '%s'" % pessoa['cor']
    cursor.execute(querycor)
    id_cor = cursor.fetchone()[0]
    
    queryts = "SELECT id FROM tb_tiposanguineo WHERE tipo = '%s'" % pessoa['tipo_sanguineo']
    cursor.execute(queryts)
    id_tiposanguineo = cursor.fetchone()[0]

    altura = float(str(pessoa['altura']).replace(',','.'))

    peso = pessoa['peso']
    
    data = "STR_TO_DATE('" + pessoa['data_nasc'] + "','%d/%m/%Y')"
    
    sexo = 'F' if pessoa['sexo'] == 'Feminino' else 'M'
    
    
    valores = (pessoa['nome'], pessoa['idade'], data, sexo, pessoa['signo'], altura, peso, id_cidade, id_cor, id_tiposanguineo)
    
    
    querypessoas = "INSERT INTO tb_pessoa (nome, idade, data_nasc, sexo, signo, altura, peso, id_cidade, id_cor, id_tiposanguineo) "
    querypessoas += "VALUES ('%s', %s, %s, '%s', '%s', %s, %s, %s, %s, %s) " % valores
    cursor.execute(querypessoas)
    mydb.commit()

Nome: Silvana Hadassa
Altura: 1.67
Peso: 50
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Thales Kaique
Altura: 1.63
Peso: 110
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Henry Nathan
Altura: 1.86
Peso: 106
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Lorenzo Vinicius
Altura: 1.87
Peso: 100
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Cauã Henrique
Altura: 1.74
Peso: 83
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Francisco Thomas
Altura: 1.91
Peso: 64
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Bianca Flávia
Altura: 1.54
Peso: 76
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Arthur Filipe
Altura: 1.91
Peso: 54
Id_Cidade: <class 'int'>
Id_Cor: <class 'int'>
Id_Tiposanguineo: <class 'int'>
Nome: Teresinha Jaquel

In [69]:
mydb.close()